In [1]:
from wield.control import SISO  
import numpy as np
import altair as alt
import pandas as pd

C:\Users\Jeff\code\wield-environment\wield-control\src\wield\control\_version.py:35: UserWarning: git base version 0.1 is different than the stored version 0.6.6
  warnings.warn(


## An open-loop transfer function, PK

In [2]:
data = pd.read_csv(r"C:/Users/Jeff/code/wield-environment/scripts/transfer_function_sysid/TF_thorlabspiezo.csv", comment="%")
data = data.drop(range(967,1024))
#print(data.dtypes)
data = data.apply(pd.to_numeric)
data = data.reindex(index=data.index[::-1])
#print(data.dtypes)
data['Freq'] = data['Freq'].apply(lambda x: x*2*np.pi) #convert Hz to rad/s
data['Mag'] = data['Mag'].apply(lambda x: 10**(x/20)) #convert dB to magnitude
data['label'] = 'Loop measured'

In [3]:
data['Complex'] = data.apply(lambda x: x.Mag*np.exp(1.0j*x.Phase*np.pi/180), axis=1)

## The controller, K, used when the transfer function was measured

In [4]:
def PIS_controller(prop_gain_dB,int_crossover_Hz,int_saturation_dB):
    int_crossover = int_crossover_Hz*2*np.pi
    int_saturation_mag = 10**(int_saturation_dB/20)
    prop_gain_mag = 10**(prop_gain_dB/20)
    return SISO.zpk([-int_crossover],[-int_crossover*prop_gain_mag/int_saturation_mag],prop_gain_mag)


K = PIS_controller(-8.0,10000,41.3)

In [5]:
K_tf = K.fresponse(f=data['Freq'].apply(lambda x: x/(2*np.pi))).tf #convert rad/s to Hz
kmag = [abs(x) for x in K_tf]
kphase = [np.angle(x)*180/np.pi for x in K_tf]
Kdf = pd.DataFrame({'Freq':data['Freq'],'Mag':kmag,'Phase':kphase})
Kdf['label'] = "Controller model"
Kdf['Complex'] = K_tf

In [6]:
Pdf = pd.DataFrame({'Freq':data['Freq']})
Pdf['label'] = "Plant"
Pdf['Complex'] = data['Complex']/Kdf['Complex']
Pdf['Mag'] = Pdf['Complex'].apply(lambda x: abs(x))
Pdf['Phase'] = Pdf['Complex'].apply(lambda x: np.angle(x)*180/np.pi)


In [7]:
def downsample_dataframe(df, column1, column2, n_samples):
    # Calculate the step size for downsampling
    step = len(df) // n_samples
    
    # Downsample the first column by taking evenly spaced samples
    downsampled_column1 = df[column1][::step].reset_index(drop=True)
    
    # Downsample the second column by taking the average of complex numbers in each window
    downsampled_column2 = []
    for i in range(0, len(df[column2]), step):
        window = df[column2][i:i+step]
        avg_complex = np.mean(window)
        downsampled_column2.append(avg_complex)

    downsampled_column2 = pd.Series(downsampled_column2)
    
    return downsampled_column1, downsampled_column2

In [8]:
F, C = downsample_dataframe(Pdf,'Freq','Complex',70)

In [9]:
downsampledf = pd.DataFrame({'Freq':F,'Complex':C})
downsampledf['Mag'] = downsampledf['Complex'].apply(lambda x: abs(x))
downsampledf['Phase'] = downsampledf['Complex'].apply(lambda x: np.angle(x)*180/np.pi)
downsampledf['label'] = 'downsampled'

In [10]:
from wield.control.AAA import tfAAA
f = downsampledf['Freq'].apply(lambda x: x/(2*np.pi)).to_numpy() #convert rad/s to Hz
c = downsampledf['Complex'].to_numpy()
barycentric = tfAAA(F_Hz=f, xfer=c)
all_z = barycentric.zeros
all_p = barycentric.poles
k = barycentric.gain
z = np.array(all_z) * (np.pi * 2) #convert Hz to rad/s
p = np.array(all_p) * (np.pi * 2) #convert Hz to rad/s
AAAfit = SISO.zpk(z,p,k)

C:\Users\Jeff\code\wield-environment\wield-control\src\wield\control\SISO\zpk.py:164: NumericalWarning: StateSpace is large (>50 states), using reduced response fiducial auditing heuristics. TODO to make this smarter
  warnings.warn(f"StateSpace is large (>{self.N_MAX_FID} states), using reduced response fiducial auditing heuristics. TODO to make this smarter", util.NumericalWarning)


In [11]:
AAAfit_tf = AAAfit.fresponse(f=data['Freq'].apply(lambda x: x/(2*np.pi))).tf #convert rad/s to Hz
AAAfitmag = [abs(x) for x in AAAfit_tf]
AAAfitphase = [np.angle(x)*180/np.pi for x in AAAfit_tf]
AAAfitdf = pd.DataFrame({'Freq':data['Freq'],'Mag':AAAfitmag,'Phase':AAAfitphase})
AAAfitdf['label'] = "AAA fit"
AAAfitdf['Complex'] = AAAfit_tf

In [12]:
from wield.iirrational.v2 import data2filter
iir_results = data2filter(
    F_Hz=f,#from a few cells ago
    xfer=c,
    mode='reduce',
    zeros=tuple(z), 
    poles=tuple(p),
    gain=k,
    SNR_phase_rel=0,
    SNR=np.ones_like(f),
    relative_degree=-4,
    # resavg_RthreshOrdDn=1.01,
    baseline_only=True,
    # coding_map=fitters_ZPK.codings_s.coding_maps.RI
    # trust_SNR = True,
    )

C:\Users\Jeff\code\wield-environment\wield-iirrational\src\wield\iirrational\_version.py:35: UserWarning: git base version 0.1 is different than the stored version 2.9.5
  warnings.warn(
C:\Users\Jeff\code\wield-environment\wield-declarative\src\wield\declarative\_version.py:35: UserWarning: git base version 1.3.6 is different than the stored version 1.5.2
  warnings.warn(


TEE_LOGFILE None
A [-2.53640587e+03     +0.j         -1.08854142e+02     +0.j
 -3.85542323e+04     +0.j          7.55276112e+02+553434.78617538j
  7.55276112e+02-553434.78617538j -5.73076841e+03+447528.31661744j
 -5.73076841e+03-447528.31661744j -4.23419737e+04+363425.13676455j
 -4.23419737e+04-363425.13676455j -1.12746304e+04+281907.83519849j
 -1.12746304e+04-281907.83519849j -3.51609926e+03+204988.6812194j
 -3.51609926e+03-204988.6812194j  -6.73068720e+03+132597.20149474j
 -6.73068720e+03-132597.20149474j  1.03104214e+04 +46189.70701728j
  1.03104214e+04 -46189.70701728j -2.38939046e+03 +32158.41869724j
 -2.38939046e+03 -32158.41869724j -9.94214232e+02 +23090.41313884j
 -9.94214232e+02 -23090.41313884j -9.68601807e+02 +18956.97910859j
 -9.68601807e+02 -18956.97910859j -6.82878413e+00  +1015.43712047j
 -6.82878413e+00  -1015.43712047j -1.39511620e+04+183283.61641801j
 -1.39511620e+04-183283.61641801j  3.84269544e+04+366933.04895544j
  3.84269544e+04-366933.04895544j -3.02628459e+04 +6

C:\Users\Jeff\code\wield-environment\wield-iirrational\src\wield\iirrational\fitters_ZPK\MRF.py:1361: RuntimeWarning: divide by zero encountered in scalar divide
  ratio = y_min / y_max
3W   0.66  Fitter_checkpoint improvement succeed, None


------------:Q-ranked order reduction:
4P   1.01    order reduced annealing


3W   1.00    Fitter_checkpoint improvement succeed, None
3W   2.38    Fitter_checkpoint improvement succeed, None
3W   7.49  Fitter_checkpoint improvement succeed, None


...

3W  30.43    Fitter_checkpoint improvement succeed, None


5P  30.43    order reduced to 11, residuals=1.51e-01, reldeg=-3


3W  33.23    Fitter_checkpoint improvement succeed, None


5P  33.23    order reduced to 10, residuals=1.46e-01, reldeg=-3
2A  35.02  Baseline fit residuals: 1.46e-01, at order 10
BASELINE:  10
------------:investigations:
2I  35.05    max(z, p)       ChiSq.
                   order    avg. res.    med. res.    max. res.
             -----------  -----------  -----------  -----------
                      10     0.145979   0.00284156       2.3183


3W  35.01  Fitter_checkpoint improvement succeed, None


In [13]:
freq = data['Freq'].apply(lambda x: x/(2*np.pi))
iirdf = pd.DataFrame({'Freq':data['Freq'],'Complex':iir_results.fitter.xfer_eval(freq)})
iirdf['Mag'] = iirdf['Complex'].apply(lambda x: abs(x))
iirdf['Phase'] = iirdf['Complex'].apply(lambda x: np.angle(x)*180/np.pi)
iirdf['label'] = 'data2filter'


In [14]:
alldata = pd.concat([data,Kdf,Pdf,downsampledf,AAAfitdf,iirdf])
alldata['Freq (Hz)'] = alldata['Freq'].apply(lambda x: x/(2*np.pi)) #convert frequencies to Hz for plotting
alldata = alldata.drop(['Complex'],axis = 1)

In [15]:
magnitudechart = alt.Chart(alldata).mark_line().encode(
    x=alt.X('Freq (Hz):Q').scale(type="log"),
    y=alt.Y('Mag:Q').scale(type="log"),
    color='label:N',
).properties(
    width=400,
    height=200
)

phasechart = alt.Chart(alldata).mark_line().encode(
    x=alt.X('Freq (Hz):Q').scale(type="log"),
    y=alt.Y('Phase:Q'),
    color='label:N',
).properties(
    width=400,
    height=100
)

chart = alt.vconcat(magnitudechart,phasechart)
chart

alt.VConcatChart(...)